# Explainer Model Performance Analysis

This notebook analyzes the performance of different explainer models in generating explanations for SAE latent features. We compare models across multiple metrics including accuracy, F1 scores, token usage, and execution time.

## Recent Updates
- **Updated to 400 latents**: Changed from 100 to 400 latents to match the experiment configuration
- **Fixed directory pattern matching**: Now handles both old (with suffix) and new (without suffix) directory naming conventions
- **Simplified loading process**: Now uses functions from `result_analysis.py` to reduce redundancy
- **Enhanced model support**: Updated model name mapping for better display names and added GPT-OSS-20B
- **Bootstrapped confidence intervals**: Error bars now use 95% CI via bootstrapping (2000 resamples) instead of standard deviation
- **Progress bars**: Added TQDM progress bars for bootstrap calculations to show progress
- **Better variable naming**: Renamed `accuracy_df` to `performance_df` for clarity

## Analysis Overview

- **Accuracy Distribution**: Density plots showing accuracy distribution for each model
- **Mean Performance**: Bar charts comparing mean accuracy across models
- **Token Usage**: Analysis of computational efficiency and resource consumption
- **Performance Summary**: Comprehensive comparison tables

## 1. Setup and Configuration

Import required libraries, define model name mapping for prettier display names, and set up output directories.

In [ ]:
import sys
import os
import json
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from scipy import stats
from tqdm.auto import tqdm

# Add the parent directory to the path to import delphi modules
sys.path.append(str(Path.cwd().parent))

from delphi.log.result_analysis import (
    import_plotly,
    load_data,
    get_agg_metrics,
    add_latent_f1,
    compute_confusion,
    compute_classification_metrics,
    frequency_weighted_f1
)

# Import plotly for plotting
px = import_plotly()

# Toggle for whether to run expensive bootstrap resampling
ENABLE_BOOTSTRAP = True  # Set to True to run bootstrap resampling (slow)
BOOTSTRAP_SAMPLES = 100  # Number of bootstrap resamples when enabled
N_JOBS = min(4, max(1, (os.cpu_count() or 1) - 1))  # Conservative parallelism for stability
# Confidence level for CI calculations (e.g. 0.95 for 95% CI). Make configurable.
CONFIDENCE_LEVEL = 0.95

# Configuration - Model prefix for experiment naming
MODEL_PREFIX = "pythiaST"  # Change this to match your base model (e.g., "pythia", "gemma", etc.)
LATENTS_COUNT = 400  # Number of latents used in experiments
COMPARE_THINKING_MODES = False  # Set to True to compare both thinking and non-thinking modes (Currently broken if set to True)

# Define model name mapping for prettier display - updated to match actual directory names
MODEL_NAME_MAPPING = {
    "gemma_3_4b_it_quantized_w4a16": "Gemma-3-4B-IT",
    "Qwen3_4B_quantized_w4a16": "Qwen3-4B", 
    "gemma_3_12b_it_quantized_w4a16": "Gemma-3-12B-IT",
    "gemma_3_27b_it_quantized_w4a16": "Gemma-3-27B-IT",
    "Qwen3_14B_quantized_w4a16": "Qwen3-14B",
    "Qwen3_32B_quantized_w4a16": "Qwen3-32B",
    "Qwen3_235B_A22B_GPTQ_Int4": "Qwen3-235B",
    "Llama_3_3_70B_Instruct_quantized_w4a16": "Llama-3.3-70B-Instruct",
    "Llama_3_1_70B_Instruct_NVFP4": "Llama-3.1-70B-Instruct",
    "Llama_4_Scout_17B_16E_Instruct_quantized_w4a16": "Llama-4-Scout-17B",
    "Llama_4_Maverick_17B_128E_Instruct_quantized_w4a16": "Llama-4-Maverick-17B",
    "llama_8b_explainer": "Transluce-Llama-8B (Qwen 32B Scorer)",
    "Meta_Llama_3_1_8B_Instruct_GPTQ_INT4": "Llama-8B (Qwen 32B Scorer)",
    "gpt_oss_20b": "GPT-OSS-20B"
}

def load_model_results(results_dir: Path, model_mapping: dict, model_prefix: str, latents_count: int, compare_modes: bool = False):
    """Load results and statistics for all explainer models using functions from result_analysis.py."""
    model_results = {}
    model_stats = {}

    # Base directory for experiments with this latent count
    experiments_base = results_dir / model_prefix / f"{latents_count}latents"

    if not experiments_base.exists():
        print(f"Warning: Experiments directory not found: {experiments_base}")
        return model_results, model_stats

    # Pattern for experiment directories (exclude thinking mode and hide directories)
    # Handle both naming conventions: with and without latents suffix
    pattern_with_suffix = f"{model_prefix}_*_{latents_count}latents"
    pattern_without_suffix = f"{model_prefix}_*"
    
    all_exp_dirs = []
    # First try directories with latents suffix (older format)
    for exp_dir in experiments_base.glob(pattern_with_suffix):
        if not ("thinking" in exp_dir.name or "hide" in exp_dir.name):
            all_exp_dirs.append(exp_dir)
    
    # Then try directories without latents suffix (newer format)
    if not all_exp_dirs:
        for exp_dir in experiments_base.glob(pattern_without_suffix):
            if (not ("thinking" in exp_dir.name or "hide" in exp_dir.name) and 
                exp_dir.name != "cache"):  # Exclude cache directory
                all_exp_dirs.append(exp_dir)

    for exp_dir in all_exp_dirs:
        # Skip directories with "thinking" or "hide" in the name
        if "thinking" in exp_dir.name or "hide" in exp_dir.name:
            continue

        # Extract model name from directory: handle both naming conventions
        dir_name = exp_dir.name
        if dir_name.endswith(f"_{latents_count}latents"):
            # Old format: pythiaST_model_400latents
            model_key = dir_name.replace(f"{model_prefix}_", "").replace(f"_{latents_count}latents", "")
        else:
            # New format: pythiaST_model
            model_key = dir_name.replace(f"{model_prefix}_", "")
        
        base_display_name = model_mapping.get(model_key, model_key)
        display_name = base_display_name

        scores_path = exp_dir / "scores"
        if scores_path.exists():
            # Load scoring results using functions from result_analysis.py
            try:
                # All experiments now use the shared cache
                cache_dir_name = "cache"
                latents_path = experiments_base.parent / cache_dir_name / "latents"
                if not latents_path.exists():
                    latents_path = exp_dir / "latents"  # Fallback to local latents

                if latents_path.exists():
                    # Extract module names from the actual files
                    sample_score_dir = next(scores_path.iterdir())
                    sample_files = list(sample_score_dir.glob("*.txt"))
                    if sample_files:
                        # Extract module name from filename pattern (e.g., "layers.32_latent0.txt" -> "layers.32")
                        sample_filename = sample_files[0].stem
                        module_name = sample_filename.split('_latent')[0]
                        modules = [module_name]
                    else:
                        print(f"No score files found in {sample_score_dir}")
                        continue

                    # Use load_data from result_analysis.py
                    latent_df, counts = load_data(scores_path, latents_path, modules)

                    if latent_df.empty:
                        print(f"No data found for {display_name}")
                        continue

                    # Use add_latent_f1 and get_agg_metrics from result_analysis.py
                    latent_df = add_latent_f1(latent_df)
                    processed_df = get_agg_metrics(latent_df, counts)

                    model_results[display_name] = {
                        'latent_df': latent_df,
                        'processed_df': processed_df,
                        'counts': counts
                    }
                else:
                    print(f"Latents path not found for {display_name}")

            except Exception as e:
                print(f"Error loading results for {model_key}: {e}")
                continue

        # Load explainer statistics
        stats_file = exp_dir / "explainer_stats.json"
        if stats_file.exists():
            try:
                with open(stats_file, 'r') as f:
                    stats = json.load(f)
                    model_stats[display_name] = stats
            except Exception as e:
                print(f"Error loading stats for {model_key}: {e}")
                model_stats[display_name] = None
        else:
            model_stats[display_name] = None

    return model_results, model_stats

# Set up directories
results_dir = Path.cwd().parent / "results"
visualizations_dir = results_dir / "visualizations"
visualizations_dir.mkdir(exist_ok=True, parents=True)

print(f"Model prefix: {MODEL_PREFIX}")
print(f"Latents count: {LATENTS_COUNT}")
print(f"Compare thinking modes: {COMPARE_THINKING_MODES}")
print(f"Results directory: {results_dir}")
print(f"Visualizations output: {visualizations_dir}")

# Check if the experiments directory exists
experiments_dir = results_dir / MODEL_PREFIX / f"{LATENTS_COUNT}latents"
print(f"Experiments directory: {experiments_dir}")

if experiments_dir.exists():
    print(f"Available experiments ({LATENTS_COUNT} latents):")
    # Handle both naming conventions: with and without latents suffix
    pattern_with_suffix = f"{MODEL_PREFIX}_*_{LATENTS_COUNT}latents"
    pattern_without_suffix = f"{MODEL_PREFIX}_*"
    
    all_exp_dirs = []
    # First try directories with latents suffix (older format)
    for exp_dir in sorted(experiments_dir.glob(pattern_with_suffix)):
        if not ("thinking" in exp_dir.name or "hide" in exp_dir.name):
            all_exp_dirs.append(exp_dir)
    
    # Then try directories without latents suffix (newer format)
    if not all_exp_dirs:
        for exp_dir in sorted(experiments_dir.glob(pattern_without_suffix)):
            if (not ("thinking" in exp_dir.name or "hide" in exp_dir.name) and 
                exp_dir.name != "cache"):  # Exclude cache directory
                all_exp_dirs.append(exp_dir)
    
    for exp_dir in all_exp_dirs:
        print(f"  - {exp_dir.name}")
else:
    print(f"Warning: Experiments directory not found: {experiments_dir}")
    print("Make sure experiments have been run with the specified latent count.")

Model prefix: pythiaST
Latents count: 400
Compare thinking modes: False
Results directory: /home/jeremias/projects/delphi-explanations/results
Visualizations output: /home/jeremias/projects/delphi-explanations/results/visualizations
Experiments directory: /home/jeremias/projects/delphi-explanations/results/pythiaST/400latents
Available experiments (400 latents):
  - pythiaST_Llama_3_3_70B_Instruct_quantized_w4a16
  - pythiaST_Llama_4_Scout_17B_16E_Instruct_quantized_w4a16
  - pythiaST_Meta_Llama_3_1_8B_Instruct_GPTQ_INT4
  - pythiaST_Qwen3_14B_quantized_w4a16
  - pythiaST_Qwen3_32B_quantized_w4a16
  - pythiaST_Qwen3_4B_quantized_w4a16
  - pythiaST_gemma_3_12b_it_quantized_w4a16
  - pythiaST_gemma_3_27b_it_quantized_w4a16
  - pythiaST_gemma_3_4b_it_quantized_w4a16
  - pythiaST_gpt_oss_20b
  - pythiaST_llama_8b_explainer


## 2. Load Model Results

Load explanation comparison results from all models and extract performance metrics.

In [2]:
# Load all model results and statistics
print("Loading model results...")
model_results, model_stats = load_model_results(results_dir, MODEL_NAME_MAPPING, MODEL_PREFIX, LATENTS_COUNT, COMPARE_THINKING_MODES)

print(f"\nLoaded results for {len(model_results)} models:")
for model_name in model_results.keys():
    print(f"  - {model_name}")

print(f"\nToken usage statistics available for {len([k for k, v in model_stats.items() if v is not None])} models:")
for model_name, stats in model_stats.items():
    if stats:
        print(f"  - {model_name}: {list(stats.keys())}")
    else:
        print(f"  - {model_name}: No stats available")

# Display sample metrics for the first model
if model_results:
    sample_model = list(model_results.keys())[0]
    sample_data = model_results[sample_model]['processed_df']
    print(f"\nSample metrics from {sample_model}:")
    print(sample_data[['score_type', 'accuracy', 'f1_score', 'precision', 'recall']].round(3))

Loading model results...


/home/jeremias/projects/delphi-explanations/delphi/log/result_analysis.py:444: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/home/jeremias/projects/delphi-explanations/delphi/log/result_analysis.py:444: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

/home/jeremias/projects/delphi-explanations/delphi/log/result_analysis.py:444: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is d


Loaded results for 11 models:
  - Qwen3-4B
  - Gemma-3-4B-IT
  - Gemma-3-12B-IT
  - Gemma-3-27B-IT
  - Qwen3-14B
  - Qwen3-32B
  - Llama-3.3-70B-Instruct
  - GPT-OSS-20B
  - Llama-8B (Qwen 32B Scorer)
  - Llama-4-Scout-17B
  - Transluce-Llama-8B (Qwen 32B Scorer)

Token usage statistics available for 11 models:
  - Qwen3-4B: ['DefaultExplainer']
  - Gemma-3-4B-IT: ['DefaultExplainer']
  - Gemma-3-12B-IT: ['DefaultExplainer']
  - Gemma-3-27B-IT: ['DefaultExplainer']
  - Qwen3-14B: ['DefaultExplainer']
  - Qwen3-32B: ['DefaultExplainer']
  - Llama-3.3-70B-Instruct: ['DefaultExplainer']
  - GPT-OSS-20B: ['DefaultExplainer']
  - Llama-8B (Qwen 32B Scorer): ['DefaultExplainer']
  - Llama-4-Scout-17B: ['DefaultExplainer']
  - Transluce-Llama-8B (Qwen 32B Scorer): ['DefaultExplainer']

Sample metrics from Qwen3-4B:
  score_type  accuracy  f1_score  precision  recall
0  detection     0.757     0.755      0.763   0.747
1       fuzz     0.818     0.827      0.787   0.871


## 3. Generate Mean F1 Score Bar Charts

Create bar charts displaying mean F1 scores with **bootstrapped 95% confidence intervals** (2000 resamples) for each model and score type. This provides more robust uncertainty estimates compared to standard deviation.

In [3]:
# Generate inline bar charts for frequency-weighted F1 scores with optional bootstrapped CI error bars and save to files
print("Preparing frequency-weighted F1 score bar charts (bootstrapping={})...".format(ENABLE_BOOTSTRAP))

from tqdm.auto import tqdm

# Helper: per-module F1s (unchanged)
def compute_per_module_f1s(score_subset, counts):
    """Return list of per-module frequency-weighted F1 values computed from the given score_subset (no resampling).
    Each module contributes a single weighted F1 computed from its latents in score_subset."""
    rows = []
    for (module, latent_idx), grp in score_subset.groupby(["module", "latent_idx"]):
        if module in counts and latent_idx < len(counts[module]):
            f1 = compute_classification_metrics(compute_confusion(grp))["f1_score"]
            fire = counts[module][latent_idx].item()
            rows.append({"module": module, "latent_idx": latent_idx, "f1_score": f1, "firing_count": fire})
    if not rows:
        return []
    df = pd.DataFrame(rows)
    per_module_vals = []
    for module in df['module'].unique():
        mdf = df[df['module'] == module]
        firing_weights = counts[module][mdf['latent_idx']].float()
        total_weight = firing_weights.sum()
        if total_weight > 0:
            f1_tensor = torch.as_tensor(mdf['f1_score'].values, dtype=torch.float32)
            module_f1 = (f1_tensor * firing_weights).sum() / firing_weights.sum()
            per_module_vals.append(module_f1.item())
    return per_module_vals

# Helper: weighted percentile (unchanged)
def weighted_percentile(values, weights, percentiles):
    """Compute weighted percentiles for 1D arrays of values and corresponding non-negative weights."""
    if len(values) == 0:
        return [np.nan for _ in np.atleast_1d(percentiles)]
    vals = np.asarray(values, dtype=float)
    w = np.asarray(weights, dtype=float)
    if w.sum() == 0:
        # Fall back to unweighted percentiles
        return np.percentile(vals, percentiles)
    # sort by value
    order = np.argsort(vals)
    vals = vals[order]
    w = w[order]
    cumsum = np.cumsum(w)
    total = cumsum[-1]
    # cumulative fraction positions of each value
    cumfrac = (cumsum - 0.5 * w) / total
    out = []
    for p in np.atleast_1d(percentiles) / 100.0:
        # find the first index where cumfrac >= p
        idx = np.searchsorted(cumfrac, p, side='left')
        if idx >= len(vals):
            out.append(vals[-1])
        else:
            out.append(vals[idx])
    return out if len(out) > 1 else out[0]

# Helper: compute CI errors from samples
def compute_ci_from_samples(samples, freq_weighted_f1, confidence):
    if not samples:
        return 0.0, 0.0
    lower_pct = (1.0 - confidence) / 2.0 * 100.0
    upper_pct = (1.0 + confidence) / 2.0 * 100.0
    lower_ci, upper_ci = np.percentile(samples, [lower_pct, upper_pct])
    return float(freq_weighted_f1 - lower_ci), float(upper_ci - freq_weighted_f1)

# Helper: single-threaded bootstrap that precomputes indices for speed
def run_bootstrap_single_thread(score_subset, counts, n_boot, confidence):
    n = len(score_subset)
    samples = []
    if n <= 0:
        return samples
    # Try to precompute indices for speed (may be memory heavy for very large n_boot * n)
    try:
        idxs = np.random.randint(0, n, size=(n_boot, n))
    except Exception:
        idxs = None
    for i in tqdm(range(n_boot), desc=f"Bootstrapping ({n_boot})", leave=False):
        if idxs is None:
            boot_idx = np.random.randint(0, n, size=n)
        else:
            boot_idx = idxs[i]
        boot_subset = score_subset.iloc[boot_idx]
        per_module_vals = compute_per_module_f1s(boot_subset, counts)
        if per_module_vals:
            samples.append(float(np.mean(per_module_vals)))
    return samples

# Main: compute_error_bars simplified to single-threaded bootstrap and modular helpers
def compute_error_bars(model_results, score_type, enable_bootstrap: bool = ENABLE_BOOTSTRAP, n_boot: int = BOOTSTRAP_SAMPLES, confidence: float = CONFIDENCE_LEVEL):
    """Compute frequency-weighted F1 and CI errors for every model for a given score_type.

    Returns a DataFrame with columns: model, frequency_weighted_f1, ci_lower_error, ci_upper_error
    """
    rows = []
    for model_name, data in model_results.items():
        processed_df = data['processed_df']
        score_row = processed_df[processed_df['score_type'] == score_type]
        if len(score_row) == 0:
            continue
        freq_weighted_f1 = score_row['weighted_f1'].iloc[0] if 'weighted_f1' in score_row.columns else None
        latent_df = data['latent_df']
        counts = data['counts']
        score_subset = latent_df[latent_df['score_type'] == score_type]
        if len(score_subset) == 0 or counts is None or freq_weighted_f1 is None:
            rows.append({'model': model_name, 'frequency_weighted_f1': freq_weighted_f1, 'ci_lower_error': 0.0, 'ci_upper_error': 0.0})
            continue

        if enable_bootstrap:
            samples = run_bootstrap_single_thread(score_subset, counts, n_boot, confidence)
            ci_lower_error, ci_upper_error = compute_ci_from_samples(samples, freq_weighted_f1, confidence)
        else:
            # Fast path: compute weighted percentiles across per-latent F1s using firing_count as weight
            vals = []
            wts = []
            for (module, latent_idx), grp in score_subset.groupby(["module", "latent_idx"]):
                if module in counts and latent_idx < len(counts[module]):
                    f1 = compute_classification_metrics(compute_confusion(grp))["f1_score"]
                    fire = counts[module][latent_idx].item()
                    vals.append(float(f1))
                    wts.append(float(fire))
            if len(vals) == 0:
                ci_lower_error = 0.0
                ci_upper_error = 0.0
            else:
                lower_pct = (1.0 - confidence) / 2.0 * 100.0
                upper_pct = (1.0 + confidence) / 2.0 * 100.0
                lower_ci, upper_ci = weighted_percentile(np.array(vals), np.array(wts), [lower_pct, upper_pct])
                lower_ci = float(lower_ci)
                upper_ci = float(upper_ci)
                ci_lower_error = float(freq_weighted_f1 - lower_ci)
                ci_upper_error = float(upper_ci - freq_weighted_f1)

        rows.append({'model': model_name, 'frequency_weighted_f1': float(freq_weighted_f1), 'ci_lower_error': ci_lower_error, 'ci_upper_error': ci_upper_error})

    df = pd.DataFrame(rows)
    return df

# Example: compute error bars for each score_type and save/use for plotting
all_score_types = set()
for data in model_results.values():
    all_score_types.update(list(data['processed_df']['score_type'].unique()))
all_score_types = sorted(list(all_score_types))

# Precompute error bar tables for reuse in other plots
error_bar_tables = {}
for st in all_score_types:
    print(f"Computing error bars for score type: {st} (bootstrap={ENABLE_BOOTSTRAP})")
    error_bar_tables[st] = compute_error_bars(model_results, st, enable_bootstrap=ENABLE_BOOTSTRAP, n_boot=BOOTSTRAP_SAMPLES, confidence=CONFIDENCE_LEVEL)

# Now create bar charts using precomputed error bars
for score_type, error_df in error_bar_tables.items():
    if error_df.empty:
        continue
    error_df = error_df.sort_values('frequency_weighted_f1', ascending=False)
    fig = px.bar(
        error_df,
        x='model',
        y='frequency_weighted_f1',
        title=f'Frequency-Weighted F1 Score by Model - {score_type.title()} ({int(CONFIDENCE_LEVEL*100)}% CI)',
        text='frequency_weighted_f1'
    )
    fig.update_traces(
        error_y=dict(
            type='data',
            symmetric=False,
            array=error_df['ci_upper_error'],
            arrayminus=error_df['ci_lower_error']
        )
    )
    fig.update_layout(
        yaxis_range=[0, 1],
        xaxis_title="Model",
        yaxis_title=f"Frequency-Weighted F1 Score ({int(CONFIDENCE_LEVEL*100)}% CI)",
        xaxis={'tickangle': 45},
        height=500
    )
    fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
    fig.show()
    output_file_pdf = visualizations_dir / f"freq_weighted_f1_bar_{score_type}.pdf"
    output_file_png = visualizations_dir / f"freq_weighted_f1_bar_{score_type}.png"
    fig.write_image(str(output_file_pdf))
    fig.write_image(str(output_file_png))
    print(f"Saved Frequency-Weighted F1 bar chart: {output_file_pdf}")

print("\nBar charts saved to", visualizations_dir)

Preparing frequency-weighted F1 score bar charts (bootstrapping=True)...
Computing error bars for score type: detection (bootstrap=True)


Bootstrapping (2000):   0%|          | 0/2000 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 5. Performance Summary

Display a summary table of F1 scores for each model and score type, including mean and standard deviation.

In [ ]:
# Create comprehensive performance summary using data from get_agg_metrics
print("Creating comprehensive performance summary...")

# Create frequency-weighted F1 score summary using processed data
summary_rows = []
for model_name, model_data in model_results.items():
    processed_df = model_data['processed_df']
    # Use the frequency-weighted F1 from the processed data (averaged across score types)
    freq_weighted_f1 = processed_df['weighted_f1'].mean() if 'weighted_f1' in processed_df.columns else None
    summary_rows.append({
        'model': model_name,
        'frequency_weighted_f1': freq_weighted_f1,
        'accuracy': processed_df['accuracy'].mean(),
        'precision': processed_df['precision'].mean(),
        'recall': processed_df['recall'].mean()
    })
summary_df = pd.DataFrame(summary_rows).set_index('model').round(3)

print("\nModel Performance Summary (Frequency-Weighted F1 Focus):")
print("=" * 60)
print(summary_df.sort_values('frequency_weighted_f1', ascending=False))

# Save summary to CSV
summary_file = visualizations_dir / "model_frequency_weighted_f1_summary.csv"
summary_df.to_csv(summary_file)
print(f"\nSummary saved to: {summary_file}")

# Best performing models by category
print("\nBest Performing Models by Category:")
print("=" * 50)
print(f"Highest Frequency-Weighted F1: {summary_df['frequency_weighted_f1'].idxmax()} ({summary_df['frequency_weighted_f1'].max():.3f})")
print(f"Highest Accuracy: {summary_df['accuracy'].idxmax()} ({summary_df['accuracy'].max():.3f})")
print(f"Highest Precision: {summary_df['precision'].idxmax()} ({summary_df['precision'].max():.3f})")
print(f"Highest Recall: {summary_df['recall'].idxmax()} ({summary_df['recall'].max():.3f})")

print(f"\nAll visualizations and summaries saved to: {visualizations_dir}")
print("\nGenerated files:")
for file in sorted(visualizations_dir.glob("*")):
    print(f"  - {file.name}")

Creating comprehensive performance summary...

Model Performance Summary (Frequency-Weighted F1 Focus):
                                      frequency_weighted_f1  accuracy  \
model                                                                   
Qwen3-32B                                             0.719     0.845   
Gemma-3-27B-IT                                        0.715     0.833   
Qwen3-4B                                              0.686     0.787   
Llama-4-Scout-17B                                     0.674     0.813   
Transluce-Llama-8B (Qwen 32B Scorer)                  0.657     0.838   
Llama-3.3-70B-Instruct                                0.651     0.842   
Gemma-3-12B-IT                                        0.642     0.788   
Qwen3-14B                                             0.627     0.813   
Llama-8B (Qwen 32B Scorer)                            0.607     0.775   
Gemma-3-4B-IT                                         0.596     0.584   
GPT-OSS-20B         